### LBH MODEL

In [5]:
ls

 Volume in drive C is Windows
 Volume Serial Number is 78C9-6226

 Directory of c:\Users\User\Documents\GitHub\CompterVisionProject_ThreatDetectionAndFaceRecognition

31/12/2024  21:38    <DIR>          .
29/12/2024  19:56    <DIR>          ..
29/12/2024  18:04                66 .gitattributes
31/12/2024  20:35    <DIR>          .ipynb_checkpoints
31/12/2024  21:32             3,292 blazeGlaze.py
29/12/2024  19:50               869 collect_image_data.py
31/12/2024  20:21             1,035 custom_yolo11.py
29/12/2024  18:59    <DIR>          CVenv
29/12/2024  19:17    <DIR>          Documents
30/12/2024  19:05    <DIR>          face_images
31/12/2024  20:37       246,111,475 lbph_face_recognizer.yml
31/12/2024  20:17             2,179 main.py
31/12/2024  16:47    <DIR>          masks_hoods_custom_model_1
31/12/2024  17:38    <DIR>          masks_hoods_custom_model_2
31/12/2024  18:02    <DIR>          masks_hoods_custom_model_3
31/12/2024  20:07    <DIR>          masks_hoods_custom_mode

In [6]:
import cv2
import os
import numpy as np

def load_images_and_labels(dataset_path):
    images, labels = [], []
    label_map = {}
    label_counter = 0

    for person_name in os.listdir(dataset_path):
        person_path = os.path.join(dataset_path, person_name)
        if not os.path.isdir(person_path):
            continue

        if person_name not in label_map:
            label_map[person_name] = label_counter
            label_counter += 1

        label = label_map[person_name]

        for image_name in os.listdir(person_path):
            image_path = os.path.join(person_path, image_name)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            if image is not None:
                images.append(image)
                labels.append(label)

    return images, np.array(labels), label_map

# Load dataset
dataset_path = "scrape_youtube/face_images"
images, labels, label_map = load_images_and_labels(dataset_path)

# Train LBPH recognizer
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.train(images, labels)

# Save LBPH model
recognizer.save("lbph_face_recognizer.yml")
print("LBPH model trained and saved.")


LBPH model trained and saved.


In [7]:
ls

 Volume in drive C is Windows
 Volume Serial Number is 78C9-6226

 Directory of c:\Users\User\Documents\GitHub\CompterVisionProject_ThreatDetectionAndFaceRecognition

31/12/2024  21:38    <DIR>          .
29/12/2024  19:56    <DIR>          ..
29/12/2024  18:04                66 .gitattributes
31/12/2024  20:35    <DIR>          .ipynb_checkpoints
31/12/2024  21:32             3,292 blazeGlaze.py
29/12/2024  19:50               869 collect_image_data.py
31/12/2024  20:21             1,035 custom_yolo11.py
29/12/2024  18:59    <DIR>          CVenv
29/12/2024  19:17    <DIR>          Documents
30/12/2024  19:05    <DIR>          face_images
31/12/2024  21:41       262,661,494 lbph_face_recognizer.yml
31/12/2024  20:17             2,179 main.py
31/12/2024  16:47    <DIR>          masks_hoods_custom_model_1
31/12/2024  17:38    <DIR>          masks_hoods_custom_model_2
31/12/2024  18:02    <DIR>          masks_hoods_custom_model_3
31/12/2024  20:07    <DIR>          masks_hoods_custom_mode

In [8]:
# Load LBPH recognizer
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read("lbph_face_recognizer.yml")

test_image = cv2.imread("C:/Users/User/Downloads/donald-trump-political-violence-qa_feat.jpg", cv2.IMREAD_GRAYSCALE)
label, confidence = recognizer.predict(test_image)

for name, id in label_map.items():
    if label == id:
        print(f"Recognized: {name} with confidence: {confidence}")
        break


Recognized: trump with confidence: 87.54378783065194


In [9]:
import cv2

# Load Haar cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

def detect_face(image_path):
    """Detects the face in an image and returns the cropped face."""
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    start_time = time()
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(50, 50))
    end_time = time()
    print("time spent identifying faces:   ", end_time - start_time)
    if len(faces) == 0:
        raise ValueError("No face detected in the image!")

    # Crop the first detected face
    x, y, w, h = faces[0]
    cropped_face = gray[y:y + h, x:x + w]
    return cropped_face


In [10]:
from time import time

In [11]:
# Load LBPH recognizer
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read("lbph_face_recognizer.yml")

KeyboardInterrupt: 

In [62]:
%%time

# Detect face and predict
test_image_path = "C:/Users/User/Downloads/donald-trump-political-violence-qa_feat.jpg"
start_time = time()
face = detect_face(test_image_path)
label, confidence = recognizer.predict(face)
end_time = time()
for name, id in label_map.items():
    # Step 3: Handle unknown faces
    if confidence < 60:
        name = "Unknown"
    if label == id:
        print(f"Recognized: {name} with confidence: {confidence}")
        break
frame_end_time = time()

print("frame check time: ",frame_end_time - start_time)

time spent identifying faces:    0.05979466438293457
Recognized: trump with confidence: 64.94098637503595
frame check time:  0.1874237060546875
CPU times: total: 219 ms
Wall time: 187 ms


In [25]:
# Detect face and predict
test_image_path = "C:/Users/User/Downloads/donald-trump-political-violence-qa_feat.jpg"
start_time = time()
face = detect_face(test_image_path)
label, confidence = recognizer.predict(face)
end_time = time()
for name, id in label_map.items():
    # Step 3: Handle unknown faces
    if confidence < 60:
        name = "Unknown"
    if label == id:
        print(f"Recognized: {name} with confidence: {confidence}")
        break
frame_end_time = time()

print("frame check time: ",frame_end_time - start_time)

NameError: name 'time' is not defined

In [65]:
# Detect face and predict
test_image_path = "C:/Users/User/Downloads/selma test.jpg"
start_time = time()
face = detect_face(test_image_path)
label, confidence = recognizer.predict(face)
end_time = time()
for name, id in label_map.items():
    # Step 3: Handle unknown faces
    if confidence < 60:
        name = "Unknown"
    if label == id:
        print(f"Recognized: {name} with confidence: {confidence}")
        break
frame_end_time = time()

print("frame check time: ",frame_end_time - start_time)

time spent identifying faces:    0.05198025703430176
Recognized: Unknown with confidence: 54.784778377745944
frame check time:  0.16557097434997559


In [ ]:
# Detect face and predict
test_image_path = "C:/Users/User/Downloads/donald-trump-political-violence-qa_feat.jpg"
start_time = time()
face = detect_face(test_image_path)
label, confidence = recognizer.predict(face)
end_time = time()
for name, id in label_map.items():
    # Step 3: Handle unknown faces
    if confidence < 40:
        name = "Unknown"
    if label == id:
        print(f"Recognized: {name} with confidence: {confidence}")
        break
frame_end_time = time()

print("frame check time: ",frame_end_time - start_time)

In [3]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
import cv2


In [2]:
pip install scikit-learn


  Using cached scikit_learn-1.6.0-cp311-cp311-win_amd64.whl.metadata (15 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.6.0-cp311-cp311-win_amd64.whl (11.1 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip show scikit-learn


Name: scikit-learn
Version: 1.6.0
Summary: A set of python modules for machine learning and data mining
Home-page: https://scikit-learn.org
Author: 
Author-email: 
License: BSD 3-Clause License

 Copyright (c) 2007-2024 The scikit-learn developers.
 All rights reserved.

 Redistribution and use in source and binary forms, with or without
 modification, are permitted provided that the following conditions are met:

 * Redistributions of source code must retain the above copyright notice, this
   list of conditions and the following disclaimer.

 * Redistributions in binary form must reproduce the above copyright notice,
   this list of conditions and the following disclaimer in the documentation
   and/or other materials provided with the distribution.

 * Neither the name of the copyright holder nor the names of its
   contributors may be used to endorse or promote products derived from
   this software without specific prior written permission.

 THIS SOFTWARE IS PROVIDED BY THE COPYR

In [4]:
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder

In [6]:
import os
import cv2
import numpy as np
from facenet_pytorch import MTCNN, InceptionResnetV1
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import torch


In [31]:
from time import time

In [40]:
%%time

# Initialize the MTCNN face detector and InceptionResnetV1 model (FaceNet)
mtcnn = MTCNN(keep_all=True)
model = InceptionResnetV1(pretrained='vggface2').eval()

# Folder where images are stored (folders named after people)
data_folder = 'C:/Users/User/Documents/GitHub/CompterVisionProject_ThreatDetectionAndFaceRecognition/scrape_youtube/face_images_balance'

# Initialize lists to hold embeddings and labels
embeddings = []
labels = []

# Loop through each person's folder (representing a person)
for person_folder in os.listdir(data_folder):
    person_folder_path = os.path.join(data_folder, person_folder)
    print(person_folder)
    if os.path.isdir(person_folder_path):
        for img_name in os.listdir(person_folder_path):
            img_path = os.path.join(person_folder_path, img_name)
            
            # Read the image
            img = cv2.imread(img_path)
            if img is None:
                continue  # Skip if the image is invalid
            
            # Detect faces using MTCNN
            faces, _ = mtcnn.detect(img)
            
            if faces is not None:
                for face in faces:
                    # Crop and align the face
                    face_tensor = mtcnn(img)
                    
                    # Get embeddings for the face
                    with torch.no_grad():
                        embedding = model(face_tensor)
                    
                    # Check the shape of the embedding before appending
                    embedding_np = embedding.squeeze().cpu().numpy()
                    
                    # Ensure embeddings are the same shape
                    if embedding_np.shape == (512,):  # Check for consistent shape
                        embeddings.append(embedding_np)
                        labels.append(person_folder)  # Use the folder name as the label



ben
cricket
cucu
eli
female_basketball
female_soccer
gordon
hillary
holly
jordan
latrell
marseca
morgan
neymar
obama
peyton
ryan
son
trump
usyk
xi
zheng
zlatan
CPU times: total: 33min 32s
Wall time: 4min 47s


In [41]:
%%time
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

CPU times: total: 0 ns
Wall time: 12.5 ms


In [42]:
%%time

embeddings = np.array(embeddings)
encoded_labels = np.array(encoded_labels)

CPU times: total: 0 ns
Wall time: 6.02 ms


In [43]:
%%time
X_train, X_test, y_train, y_test = train_test_split(embeddings, encoded_labels, test_size=0.2, random_state=42)


CPU times: total: 0 ns
Wall time: 3.56 ms


In [44]:
%%time

svm = SVC(kernel='linear', probability=True)
print("first bit done")
svm.fit(X_train, y_train)

first bit done
CPU times: total: 406 ms
Wall time: 511 ms


SVC(kernel='linear', probability=True)

In [45]:
%%time

# Evaluate the classifier on the test set
accuracy = svm.score(X_test, y_test)
print(f"Model accuracy: {accuracy * 100:.2f}%")

Model accuracy: 100.00%
CPU times: total: 78.1 ms
Wall time: 130 ms


In [ ]:
joblib.dump(svm, 'svm_model_dummy_categories_balanced.pkl')

### category model with unbalanced for target faces

In [37]:
%%time
b # breaks incase running by accident
# Evaluate the classifier on the test set
accuracy = svm.score(X_test, y_test)
print(f"Model accuracy: {accuracy * 100:.2f}%")

Model accuracy: 99.88%
CPU times: total: 78.1 ms
Wall time: 175 ms


In [17]:
import joblib


In [38]:
joblib.dump(svm, 'svm_model_dummy_categories.pkl')

['svm_model_dummy_categories.pkl']

ValueError: Expected 2D array, got 1D array instead:
array=[1. 3. 5. 5.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.